In [1]:
import json
from datetime import datetime

In [4]:
pwd

'/home/jay.je/IMspiredStoryTelling/notebooks'

In [5]:
annotation=json.load(open('/home/jay.je/IMspiredStoryTelling/datasets/VIST/test.story-in-sequence.json'))

In [6]:
annotation.keys()

dict_keys(['images', 'info', 'albums', 'type', 'annotations'])

In [7]:
len(annotation['images'])

21075

In [8]:
annotation['images'][0]

{'datetaken': '2004-11-27 10:40:46',
 'license': '1',
 'title': 'The venue.',
 'text': '',
 'album_id': '44277',
 'longitude': '-0.212688',
 'url_o': 'https://farm1.staticflickr.com/2/1741642_81837e8e9e_o.jpg',
 'secret': '81837e8e9e',
 'media': 'photo',
 'latitude': '51.920449',
 'id': '1741642',
 'tags': 'stevenage fairies craftfair xmas 2004'}

In [9]:
annotation['albums'][0]

{'description': "I left Stevenage over twenty years ago, having lived there between the ages of 7 and 20. Yesterday (on International Buy Nothing Day!) we returned for a 'Victorian Craft Fair' in the Church above the town Museum. Very strange...",
 'title': 'A trip to Stevenage',
 'farm': '1',
 'date_update': '1396811566',
 'primary': '1741611',
 'server': '2',
 'date_create': '44277',
 'photos': '23',
 'secret': '3f374c96e5',
 'owner': '37996585435@N01',
 'vist_label': 'fair',
 'id': '44277'}

In [10]:
annotation['annotations'][0]

[{'original_text': 'The local parish holds a craft show each year.',
  'album_id': '44277',
  'photo_flickr_id': '1741642',
  'setting': 'first-2-pick-and-tell',
  'worker_id': 'FJROI8NWDRIPAM1',
  'story_id': '45530',
  'tier': 'story-in-sequence',
  'worker_arranged_photo_order': 0,
  'text': 'the local parish holds a craft show each year .',
  'storylet_id': '227650'}]

## Create "Sents" (sentences from annotations)

In [11]:
sents = []
for ann in annotation['annotations']:
	# sent = {album_id, img_id, story_id, text, original_text, }
	sent = ann[0].copy()
	sent['id'] = sent.pop('storylet_id')
	sent['order'] = sent.pop('worker_arranged_photo_order')
	sent['img_id'] = sent.pop('photo_flickr_id')
	sent['length'] = len(sent['text'].split())  # add length property
	sents += [sent]


In [12]:
annotation['annotations'][0][0]

{'original_text': 'The local parish holds a craft show each year.',
 'album_id': '44277',
 'photo_flickr_id': '1741642',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'worker_arranged_photo_order': 0,
 'text': 'the local parish holds a craft show each year .',
 'storylet_id': '227650'}

In [13]:
sents[0]

{'original_text': 'The local parish holds a craft show each year.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'the local parish holds a craft show each year .',
 'id': '227650',
 'order': 0,
 'img_id': '1741642',
 'length': 10}

## Get Album id to image id mapping

In [14]:
Albums = {album['id']: album for album in annotation['albums']}
Images = {img['id']: img for img in annotation['images']}
Sents = {sent['id']: sent for sent in sents}

In [15]:
def getDateTime(img_id):
	x = Images[img_id]['datetaken']
	return datetime.strptime(x, '%Y-%m-%d %H:%M:%S')

In [16]:
album_to_img_ids = {}
for img in annotation['images']:
	album_id = img['album_id']
	img_id = img['id']
	album_to_img_ids[album_id] = album_to_img_ids.get(album_id, []) + [img_id]
for album_id, img_ids in album_to_img_ids.items():
	img_ids.sort(key=getDateTime)

## Get story id -> Sent ids

In [18]:
def get_order(sent_id):
	return Sents[sent_id]['order']

In [19]:
story_to_sent_ids = {}
for sent_id, sent in Sents.items():
	story_id = sent['story_id']
	story_to_sent_ids[story_id] = story_to_sent_ids.get(story_id, []) + [sent_id]
for story_id, sent_ids in story_to_sent_ids.items():
	sent_ids.sort(key=get_order)

In [20]:
story_to_sent_ids # story id to sent id

{'45530': ['227650', '227651', '227652', '227653', '227654'],
 '45531': ['227655', '227656', '227657', '227658', '227659'],
 '45532': ['227660', '227661', '227662', '227663', '227664'],
 '45533': ['227665', '227666', '227667', '227668', '227669'],
 '45534': ['227670', '227671', '227672', '227673', '227674'],
 '45535': ['227675', '227676', '227677', '227678', '227679'],
 '45536': ['227680', '227681', '227682', '227683', '227684'],
 '45537': ['227685', '227686', '227687', '227688', '227689'],
 '45538': ['227690', '227691', '227692', '227693', '227694'],
 '45539': ['227695', '227696', '227697', '227698', '227699'],
 '45540': ['227700', '227701', '227702', '227703', '227704'],
 '45541': ['227705', '227706', '227707', '227708', '227709'],
 '45542': ['227710', '227711', '227712', '227713', '227714'],
 '45543': ['227715', '227716', '227717', '227718', '227719'],
 '45544': ['227720', '227721', '227722', '227723', '227724'],
 '45545': ['227725', '227726', '227727', '227728', '227729'],
 '45546'

## Album id -> Story id

In [21]:
album_to_story_ids = {}
for story_id, sent_ids in story_to_sent_ids.items():
	sent = Sents[sent_ids[0]]
	album_id = sent['album_id']
	album_to_story_ids[album_id] = album_to_story_ids.get(album_id, []) + [story_id]

In [25]:
display(Sents['227650'])
display(Sents['227651'])
display(Sents['227652'])
display(Sents['227653'])
display(Sents['227654'])
display(Sents['227655']) # this one is different story id

{'original_text': 'The local parish holds a craft show each year.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'the local parish holds a craft show each year .',
 'id': '227650',
 'order': 0,
 'img_id': '1741642',
 'length': 10}

{'original_text': 'Lots of folks come out and set up tables to sell their crafts.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'lots of folks come out and set up tables to sell their crafts .',
 'id': '227651',
 'order': 1,
 'img_id': '1741640',
 'length': 14}

{'original_text': 'Some of these crafts are very unique and take a lot of talent to make.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'some of these crafts are very unique and take a lot of talent to make .',
 'id': '227652',
 'order': 2,
 'img_id': '1741632',
 'length': 16}

{'original_text': 'Folks of all ages come out to peruse the crafts for sale.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'folks of all ages come out to peruse the crafts for sale .',
 'id': '227653',
 'order': 3,
 'img_id': '1741622',
 'length': 13}

{'original_text': 'Some of the crafters even dress up in unique costumes as part of their selling act.',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'FJROI8NWDRIPAM1',
 'story_id': '45530',
 'tier': 'story-in-sequence',
 'text': 'some of the crafters even dress up in unique costumes as part of their selling act .',
 'id': '227654',
 'order': 4,
 'img_id': '1741587',
 'length': 17}

{'original_text': 'I was so excited to be heading to the crafts fair. ',
 'album_id': '44277',
 'setting': 'first-2-pick-and-tell',
 'worker_id': 'Y6ZNXABP31FUVQ8',
 'story_id': '45531',
 'tier': 'story-in-sequence',
 'text': 'i was so excited to be heading to the crafts fair .',
 'id': '227655',
 'order': 0,
 'img_id': '1741625',
 'length': 12}

In [26]:
album_to_story_ids # each album has set of stories, and set of stories map to sent id

{'44277': ['45530', '45531', '45532', '45533', '45534'],
 '8139': ['45535', '45536', '45537', '45538', '45539'],
 '504823': ['45540', '45541', '45542', '45543', '45544'],
 '72157607155047588': ['45545', '45546', '45547', '45548', '45549'],
 '72157605016116512': ['45550', '45551', '45552', '45553', '45554'],
 '72157644777317969': ['45555', '45556', '45557', '45558', '45559'],
 '72157605629519358': ['45560', '45561', '45562', '45563', '45564'],
 '72157605630785895': ['45565', '45566', '45567', '45568', '45569'],
 '72157605638688643': ['45570', '45571', '45572', '45573', '45574'],
 '72157605635232532': ['45575', '45576', '45577', '45578', '45579'],
 '72157644824250557': ['45580', '45581', '45582', '45583', '45584'],
 '72157625498117651': ['45585', '45586', '45587', '45588', '45589'],
 '72157626882487487': ['45590', '45591', '45592', '45593', '45594'],
 '72157606307817709': ['45595', '45596', '45597', '45598', '45599'],
 '72157611736381187': ['45600', '45601', '45602', '45603', '45604'],
 

In [27]:
for album in annotation['albums']:
	album['img_ids'] = album_to_img_ids[album['id']]
	album['story_ids'] = album_to_story_ids[album['id']]

In [28]:
annotation['albums'][1]

{'description': 'Our visit on September 6th with Mary, Gil, and Diego.',
 'title': 'Albemarle County Fair, 2004',
 'farm': '1',
 'date_update': '1397505294',
 'primary': '355132',
 'server': '1',
 'date_create': '8139',
 'photos': '14',
 'secret': 'ea2d93502d',
 'owner': '36221226@N00',
 'vist_label': 'fair',
 'id': '8139',
 'img_ids': ['355132',
  '355134',
  '355135',
  '355203',
  '355137',
  '355139',
  '355142',
  '355204',
  '355205',
  '355206',
  '355208',
  '355209',
  '355331',
  '355332'],
 'story_ids': ['45535', '45536', '45537', '45538', '45539']}

In [31]:
story_to_sent_ids['45535']

['227675', '227676', '227677', '227678', '227679']

In [32]:
story_to_sent_ids['45536']

['227680', '227681', '227682', '227683', '227684']

In [33]:
for album in annotation['albums']:
	album['img_ids'] = album_to_img_ids[album['id']]
	album['story_ids'] = album_to_story_ids[album['id']]

# make Stories: {story_id: {id, album_id, sent_ids, img_ids}}
Stories = {story_id: {
                # 'id': story_id, 
                'sent_ids': sent_ids, 
                'img_ids': [Sents[sent_id]['img_id'] for sent_id in sent_ids],
                'album_id': Sents[sent_ids[0]]['album_id']} 
			for story_id, sent_ids in story_to_sent_ids.items()}

In [34]:
Stories

{'45530': {'sent_ids': ['227650', '227651', '227652', '227653', '227654'],
  'img_ids': ['1741642', '1741640', '1741632', '1741622', '1741587'],
  'album_id': '44277'},
 '45531': {'sent_ids': ['227655', '227656', '227657', '227658', '227659'],
  'img_ids': ['1741625', '1741640', '1741639', '1741633', '1741630'],
  'album_id': '44277'},
 '45532': {'sent_ids': ['227660', '227661', '227662', '227663', '227664'],
  'img_ids': ['1741642', '1741640', '1741632', '1741622', '1741587'],
  'album_id': '44277'},
 '45533': {'sent_ids': ['227665', '227666', '227667', '227668', '227669'],
  'img_ids': ['1741625', '1741640', '1741639', '1741633', '1741630'],
  'album_id': '44277'},
 '45534': {'sent_ids': ['227670', '227671', '227672', '227673', '227674'],
  'img_ids': ['1741642', '1741640', '1741632', '1741622', '1741587'],
  'album_id': '44277'},
 '45535': {'sent_ids': ['227675', '227676', '227677', '227678', '227679'],
  'img_ids': ['355137', '355139', '355142', '355331', '355332'],
  'album_id': '

## Manually download images, because one split do not have consecutive images

In [35]:
import requests
from PIL import Image
import random
import os
import shutil
import urllib.request 

from requests.adapters import HTTPAdapter, Retry

In [37]:
# Download 
len(Stories.keys()) # 5000 stories -> choose 1000 stories. Train has chosen 2000 stories

5055

In [48]:
ridx = random.sample(range(int(min(Stories)), int(max(Stories))), 1000) # collect about 1000 stories, corresponding to 5,000 images

In [49]:
img_list = []
stories = []
for i in ridx:
    if str(i) in Stories:
        stories.append(str(i))
        img_list += Stories[str(i)]['img_ids']

In [50]:
print(len(stories))
print(len(img_list))
print(img_list[0])

1000
5000
3575862648


In [51]:
Images['3575862648']

{'datetaken': '2009-05-28 19:32:31',
 'license': '1',
 'title': 'DSC_0013',
 'text': '',
 'album_id': '72157618984682634',
 'longitude': '-85.656538',
 'url_o': 'https://farm4.staticflickr.com/3400/3575862648_47a6a18ae5_o.jpg',
 'secret': '38de8735ac',
 'media': 'photo',
 'latitude': '42.437140',
 'id': '3575862648',
 'tags': 'blue white football stadium walk graduation ceremony highschool procession phs graduate congratulations trojans classof2009 plainwell nikond40'}

In [54]:
def make_request(url):
    session = requests.Session()
    retry = Retry(connect=3, backoff_factor=0.5)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)

    response = session.get(url)

    return response
    # print(parsed)

In [55]:
if os.path.exists('/home/jay.je/datasets/VIST/images/test'):
    shutil.rmtree('/home/jay.je/datasets/VIST/images/test')
os.makedirs('/home/jay.je/datasets/VIST/images/test')

In [56]:
for idx, img_num in enumerate(img_list):
    # url = Images[img_num]['url_o']
    # urllib.request.urlretrieve(url, f'/home/jay.je/datasets/VIST/images/{img_num}.jpg')
    try:
        data = requests.get(Images[img_num]['url_o'], timeout=1).content
        with open(f'/home/jay.je/datasets/VIST/images/test/{img_num}.jpeg', 'wb') as handler:
            handler.write(data)
            handler.close()
    except:
        continue
    if idx % 500 == 0:
        print(f'Current processing number is {idx}...\n')

Current processing number is 0...

Current processing number is 500...

Current processing number is 1000...

Current processing number is 1500...

Current processing number is 2000...

Current processing number is 2500...

Current processing number is 3000...

Current processing number is 3500...

Current processing number is 4000...

Current processing number is 4500...



In [111]:
# check which stories have incomplete images

In [57]:
img_down = os.listdir('/home/jay.je/datasets/VIST/images/test/')

In [58]:
print(len(img_down))
print(len(set(img_down)))

3811
3811


In [59]:
img_down = set(img_down)

In [60]:
img_down

{'16167408922.jpeg',
 '4431940282.jpeg',
 '5806469750.jpeg',
 '5224734982.jpeg',
 '2408632.jpeg',
 '17461182.jpeg',
 '8539173488.jpeg',
 '17496382.jpeg',
 '4283995394.jpeg',
 '18256518.jpeg',
 '4306138750.jpeg',
 '93462929.jpeg',
 '35433230.jpeg',
 '5690936507.jpeg',
 '1794025049.jpeg',
 '1813723349.jpeg',
 '4947302832.jpeg',
 '12844709524.jpeg',
 '156090335.jpeg',
 '16763441.jpeg',
 '6366790995.jpeg',
 '133241105.jpeg',
 '4970846212.jpeg',
 '57376587.jpeg',
 '118630865.jpeg',
 '4793228542.jpeg',
 '80260314.jpeg',
 '6230555210.jpeg',
 '68130501.jpeg',
 '37115053.jpeg',
 '4416709241.jpeg',
 '13277321174.jpeg',
 '6085978629.jpeg',
 '7350422846.jpeg',
 '4726595156.jpeg',
 '4410646.jpeg',
 '1384859697.jpeg',
 '8333610231.jpeg',
 '3686909653.jpeg',
 '5753018756.jpeg',
 '4417331961.jpeg',
 '4976768910.jpeg',
 '6498380663.jpeg',
 '4495556511.jpeg',
 '4266261926.jpeg',
 '21409422.jpeg',
 '1168126400.jpeg',
 '11221941546.jpeg',
 '450820.jpeg',
 '4249390336.jpeg',
 '4423338967.jpeg',
 '19789842.

In [61]:
drop_stories=[]
for i in Stories:
    img_ids = Stories[i]['img_ids']
    for j in img_ids:
        if j+'.jpeg' not in img_down:
            drop_stories.append(i)
            break

In [62]:
drop_stories = set(drop_stories)
print(len(drop_stories))

2786


In [65]:
set(Stories.keys())-drop_stories

{'48910',
 '50515',
 '49195',
 '46418',
 '45882',
 '49109',
 '49038',
 '47655',
 '46430',
 '48894',
 '49391',
 '49099',
 '46928',
 '48032',
 '46654',
 '50067',
 '49163',
 '50262',
 '49033',
 '46261',
 '48779',
 '45793',
 '48943',
 '47167',
 '48367',
 '46413',
 '47611',
 '49803',
 '49137',
 '46094',
 '48303',
 '46402',
 '49476',
 '48149',
 '46693',
 '49198',
 '47062',
 '46070',
 '46926',
 '49119',
 '45746',
 '49157',
 '49616',
 '46677',
 '48659',
 '47357',
 '48451',
 '48834',
 '48038',
 '49965',
 '46237',
 '45892',
 '47639',
 '47523',
 '47872',
 '48085',
 '49089',
 '47218',
 '47307',
 '48875',
 '46696',
 '48433',
 '48718',
 '50019',
 '47860',
 '46274',
 '47180',
 '47864',
 '50164',
 '48625',
 '47699',
 '45557',
 '48290',
 '49063',
 '46707',
 '48033',
 '50087',
 '48489',
 '49935',
 '46042',
 '48624',
 '46104',
 '45998',
 '46937',
 '48810',
 '46048',
 '47792',
 '49568',
 '47426',
 '47658',
 '50513',
 '48732',
 '45767',
 '46046',
 '50374',
 '49633',
 '45832',
 '46342',
 '48587',
 '46382',


In [66]:
len(set(Stories.keys())-drop_stories)

2269

In [67]:
TestStoriesFin = {}
for i in Stories:
    if i not in drop_stories:
        TestStoriesFin[i] = Stories[i].copy()

In [68]:
len(TestStoriesFin.keys()) # able to extract 2269 stories from this!

2269

In [71]:
for s in TestStoriesFin:
    for i in TestStoriesFin[s]['sent_ids']:
        if 'text' not in TestStoriesFin[s]:
            TestStoriesFin[s]['text'] = []
        TestStoriesFin[s]['text'].append(Sents[i]['text'])

In [74]:
TestStoriesFin['45531']

{'sent_ids': ['227655', '227656', '227657', '227658', '227659'],
 'img_ids': ['1741625', '1741640', '1741639', '1741633', '1741630'],
 'album_id': '44277',
 'text': ['i was so excited to be heading to the crafts fair .',
  'when i arrived i saw a great booth with a variety of great crafts .',
  "i stopped at chatted at my friend [female] 's booth for a bit .",
  'there were even booths set up for all of the kids .',
  "i found some awesome crafts at the fair , i 'm really happy that i went ."]}

## Save the final files

In [76]:
sents_json = json.dumps(Sents, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/annotations/TestSents.json", "w") as outfile:
    outfile.write(sents_json)
    outfile.close()

In [78]:
storiesfin_json = json.dumps(TestStoriesFin, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/annotations/TestStoriesFin.json", "w") as outfile:
    outfile.write(storiesfin_json)
    outfile.close()

In [79]:
stories_json = json.dumps(Stories, indent=4)
# Writing to sample.json
with open("/home/jay.je/IMspiredStoryTelling/datasets/VIST/annotations/TestStories.json", "w") as outfile:
    outfile.write(stories_json)
    outfile.close()